In [11]:
#!pip install sentence_transformers
import os
import openai

#openai.api_key  = os.environ['OPENAI_API_KEY']

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key  = os.environ['OPENAI_API_KEY']

In [1]:
from gradio_client import Client

client = Client("https://ysharma-explore-llamav2-with-tgi.hf.space/")
result = client.predict(
				"Howdy!",	# str in 'Message' Textbox component
				api_name="/chat"
)
print(result)

c:\Users\anton_glfk00f\miniconda3\envs\llm_app_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
 Hello


In [8]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

reader = PdfReader("PROGRAMA ANALITICO MÓDULO.pdf")
#reader = PdfReader("PatekPhilippe_P2400_Manual_Winding.pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")

text = ""
i = 0

for page in reader.pages:
    i += 1
    text += f"Página no. {i}: " + page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    add_start_index = True,
    )
chunks = text_splitter.split_text(text=text)

In [1]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

reader = PdfReader("PROGRAMA ANALITICO MÓDULO.pdf")
#reader = PdfReader("PatekPhilippe_P2400_Manual_Winding.pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
VectorStore = FAISS.from_texts([""], embedding=embeddings)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    add_start_index = False,
    )
i = 0
for page in reader.pages:
    i += 1
    text = page.extract_text()
    #print(text)
    chunks = text_splitter.split_text(text=text)
    chunks_pages = []
    for chunk in chunks:
        chunk_wpage = f"Página no. {i} " + chunk
        chunks_pages.append(chunk_wpage)
    print(chunks_pages)
    VectorStore.add_texts(chunks_pages)

c:\Users\anton_glfk00f\miniconda3\envs\llm_app_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['Página no. 1 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   \n  E-mail: solicitudes@uagrmbs.edu.bo   \n  \n   ESPECIFICACION  \n \nREQUISITOS ACADEMICOS PARA LA REALIZACION DE  \nUN  MODULO  \nCódigo: ES -ENA -551  Rev.   6, Fecha:  23-03-12  \nPág.  1 de 4  \n  \n   \n \n  \nUAGRM BUSINESS SCHOOL      \n  \n  \n \n  \n   \n  \n  \nMAESTRÍA:  \nMAESTRÍA EN ADMINISTRACIÓN DE EMPRESAS  \n(Versión 9º – Edición 10º)  \nGESTIÓN DE MARKETING   \n(ESTRATÉGICO Y OPERATIVO)  \n  \n  \n \n  \nEnero de 2022  \nSanta Cruz – Bolivia  \n \n \nANEXO  \n  \n1 \n: \n  \nFORMATO DE  \nPRESENTACION DEL PROGRAMA ANAL  \nI \nTICO  \n:']
['Página no. 2 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   \n  E-mail: solicitudes@uagrmbs.edu.bo   \n  \n   ESPECIFICACION  \n \nREQUISITOS ACADEMICOS PARA LA REALIZACION DE  \nUN  MODULO  \nCódigo: ES -ENA -551  Rev.   6, Fecha:  23-03-12  \nPág.  2 de 4 

In [88]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
VectorStore = FAISS.from_texts(chunks, embedding=embeddings)

In [24]:
import pickle

# Save the object to a file using pickle
file_name = 'VectorStore_Marketing_en.pickle'
with open(file_name, 'wb') as file:
    pickle.dump(VectorStore, file)


In [25]:
import pickle

# Load the object from the file using pickle
with open('VectorStore_Marketing.pickle', 'rb') as file:
    loaded_VectorStore = pickle.load(file)

In [3]:
from gpt4all import GPT4All
model = GPT4All("llama-2-7b-chat.ggmlv3.q4_0.bin")
output = model.generate("The capital of France is ", max_tokens=30)
print(output)

Found model file at  C:\\\\Users\\\\anton_glfk00f\\\\.cache\\\\gpt4all\\llama-2-7b-chat.ggmlv3.q4_0.bin
 Paris, which is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre


In [40]:
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from gradio_client import Client

class LLM_gradioAPI(LLM):
    n: int

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        # Remove the restriction on the stop parameter
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")


        # Make the API call using the Gradio Client
        client = Client("https://ysharma-explore-llamav2-with-tgi.hf.space/")
        result = client.predict(prompt, api_name="/chat_1")

        # Return the response from the API
        return result

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}

In [18]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from langchain.llms import GPT4All
from src import LLM_gradioAPI

query = "Responde en español: ¿Cuál es la experiencia  de la docente?"
docs = VectorStore.similarity_search(query=query, k=3)
#docs = VectorStore.max_marginal_relevance_search(query=query, k=5)

#llm = OpenAI()
#llm = GPT4All(model="./llama-2-7b-chat.ggmlv3.q4_0.bin",max_tokens=2048)
llm = LLM_gradioAPI(n=100) # hugging face repo
chain = load_qa_chain(llm=llm, chain_type="stuff", verbose=True)

reference = "------Also, Write in new lines the initial 10 words of each context for reference."
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=query)
    print(cb)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Página no. 2 Experiencia en el área comercial y ventas de empresas de servicio como Principal Créditos Hipotecarios, 
Andueza Corredores de Seguros, Santiago -Chile. Gerencia de Ventas  de Joyerías Anahí, Santa 
CruzBolivia. Consultoría en temas empresariales y de marketing. Docencia en pre y postgrado  en UTEPSA, 
UAGRM Business School y UCB.  
  
II. PRESENTACION DEL MODULO  
El módulo Gestión de Marketing entrega la metodología y las herramientas para entender el mercado en el 
cual está compiten las  empresas, comprendiendo la importancia del entorno y de las habilidades de la gestión 
de marketing para aprovechas las oportunid ades y defenderse de las amenazas.   
El postgraduante podrá incrementar su habilidad de a

In [19]:
response

'La docente tiene experiencia en el área comercial y ventas de empresas de servicio, como Principal Créditos Hipotecarios y Andueza Corredores de Seguros, en Santiago, Chile. También ha trabajado en la gerencia de ventas de Joyerías Anahí en Santa Cruz, Bolivia. Además, ha sido consultora en temas empresariales y de marketing, y ha impartido clases en pre y postgrados en UTEPSA, UAGRM Business School y UCB.'

In [106]:
response

' The teacher has experience in marketing management and sales of financial institutions, credit agencies, Santiago - Chile. Marketing consulting services, business school and UCB.'

In [33]:
response

' Las estrategias del marketing incluyen estrategias específicas para mercados meta, posicionamiento, la mezcla de marketing y los niveles de los gastos de marketing.'

In [26]:
response

' El marketing es el proceso de satisfacer las necesidades de los consumidores mediante la identificación, anticipación y satisfacción de sus deseos. Esto implica determinar qué productos y servicios los consumidores desean, establecer un precio adecuado para ellos, promocionarlos adecuadamente y distribuirlos en el lugar correcto. El marketing involucra también el estudio de la competencia y el análisis de la industria.'

In [21]:
response

' El marketing es el proceso de diseñar relaciones redituables con el cliente mediante la creación de valor para los clientes y la captación de valor de éstos a cambio. Se enfoca en la creación de valor para los clientes a través de la identificación y satisfacción de sus necesidades. Esto se lleva a cabo mediante la promoción, ventas y distribución de productos y servicios.'

### Chain

In [47]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings # see also sentence-transformers
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from src import LLM_gradioAPI

reader = PdfReader("PROGRAMA ANALITICO MÓDULO.pdf")
#reader = PdfReader("PatekPhilippe_P2400_Manual_Winding.pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
VectorStore = FAISS.from_texts([""], embedding=embeddings)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    add_start_index = False,
    )
i = 0
for page in reader.pages:
    i += 1
    text = page.extract_text()
    #print(page)
    chunks = text_splitter.split_text(text=text)
    chunks_pages = []
    for chunk in chunks:
        chunk_wpage = f"Página no. {i} " + chunk
        chunks_pages.append(chunk_wpage)
    print(chunks_pages)
    VectorStore.add_texts(chunks_pages)

['Página no. 1 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   \n  E-mail: solicitudes@uagrmbs.edu.bo   \n  \n   ESPECIFICACION  \n \nREQUISITOS ACADEMICOS PARA LA REALIZACION DE  \nUN  MODULO  \nCódigo: ES -ENA -551  Rev.   6, Fecha:  23-03-12  \nPág.  1 de 4  \n  \n   \n \n  \nUAGRM BUSINESS SCHOOL      \n  \n  \n \n  \n   \n  \n  \nMAESTRÍA:  \nMAESTRÍA EN ADMINISTRACIÓN DE EMPRESAS  \n(Versión 9º – Edición 10º)  \nGESTIÓN DE MARKETING   \n(ESTRATÉGICO Y OPERATIVO)  \n  \n  \n \n  \nEnero de 2022  \nSanta Cruz – Bolivia  \n \n \nANEXO  \n  \n1 \n: \n  \nFORMATO DE  \nPRESENTACION DEL PROGRAMA ANAL  \nI \nTICO  \n:']
['Página no. 2 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   \n  E-mail: solicitudes@uagrmbs.edu.bo   \n  \n   ESPECIFICACION  \n \nREQUISITOS ACADEMICOS PARA LA REALIZACION DE  \nUN  MODULO  \nCódigo: ES -ENA -551  Rev.   6, Fecha:  23-03-12  \nPág.  2 de 4 

In [52]:
query = "Responde en español: ¿Qué es el marketing?"
docs = VectorStore.similarity_search(query=query, k=3)
#docs = loaded_VectorStore.max_marginal_relevance_search(query=query, k=5)

llm = LLM_gradioAPI(n=100) # hugging face repo
chain = load_qa_chain(llm=llm, chain_type="stuff", verbose=True)

reference = "------Also, Write in new lines the initial 10 words of each context for reference."
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=query)
    print(cb)
print(response)

KeyError: 'page'

### Using langcahin pypdf loader

In [76]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings # see also sentence-transformers
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from src import LLM_gradioAPI
from langchain.document_loaders import PyPDFLoader

#reader = PdfReader("PROGRAMA ANALITICO MÓDULO.pdf")
#reader = PdfReader("PatekPhilippe_P2400_Manual_Winding.pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#reader = PdfReader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")

#loader = PyPDFLoader("PROGRAMA ANALITICO MÓDULO.pdf")
loader = PyPDFLoader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")


pages = loader.load_and_split()

chunks = text_splitter.split_documents(pages)


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
faiss_index = FAISS.from_documents(chunks, embedding=embeddings)


Referencias:

Página no.: 17
 xvi Prefacio
CENTRO DE RECURSOS PARA EL INSTRUCTOR: Todos los recursos para el
instructor están en inglés y protegidos por una clave, y están disponibles para su descarga en
www.pearsoneducacion.net/kotler . Para su comodidad, estos recursos también están dispo-
nibles en el CD-ROM de Recursos para el Profesor.
■Manual del instructor :Revise este recurso capítulo por capítulo o descargue todo el ma-
nual en un archivo comprimido.
■Archivo de reactivos para examen :Revise capítulo por capítulo o descargue todo el 
archivo de reactivos para examen en un archivo comprimido.
■TestGen para PC/Mac: Descargue este programa fácil de usar; ya viene incluido en las
preguntas de examen y manual del usuario de la decimoprimera edición.
■Banco de imágenes (sólo en CD) : Tenga acceso a muchas de las imágenes, anuncios e
ilustraciones incluidos en el texto. Ideal para presentaciones con PowerPoint.
■Presentaciones en PowerPoint :Cuando se trata de presentaciones en Power

In [88]:
docs = faiss_index.similarity_search("¿Qué es marketing?", k=5)
#docs = loaded_VectorStore.max_marginal_relevance_search(query=query, k=5)

llm = LLM_gradioAPI(n=2000) # hugging face repo
chain = load_qa_chain(llm=llm, chain_type="stuff", verbose=False)
response = chain.run(input_documents=docs, question=query)
#with get_openai_callback() as cb:
#    response = chain.run(input_documents=docs, question=query)
#    print(cb)

print(response)
print("\nReferencias:")
for doc in docs:
    print("\nPágina no.: " + str(doc.metadata["page"]) + "\n", doc.page_content[:150] + "...")

Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
El marketing es un proceso social y administrativo mediante el cual los individuos y los grupos obtienen lo que necesitan y desean, creando e intercambiando valor con otros. En el contexto de negocios, el marketing incluye el desarrollo, la producción, la distribución, la promoción y la venta de productos o servicios que satisfagan las necesidades y deseos de los clientes. El marketing directo, por ejemplo, es una forma de marketing que se enfoca en individuos específicos y utiliza cuatro características distintivas: no es público, es inmediato y personalizado, y es interactivo. En resumen, el marketing es una estrategia utilizada para establecer relaciones con los clientes y satisfacer sus necesidades, lo que puede incluir la venta y la promoción de productos o servicios.

Referencias:

Página no.: 25
 Definición de marketing
¿Qué esel marketing? Mucha gente cree que el marketing sólo consiste en vender y hacer pu-
bl

### A draft

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings # see also sentence-transformers
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from src import LLM_gradioAPI
from langchain.document_loaders import PyPDFLoader # install pypdf
from langchain.prompts import PromptTemplate

loader = PyPDFLoader("PROGRAMA ANALITICO MÓDULO.pdf")
#loader = PyPDFLoader("PROGRAMA ANALITICO MÓDULO.pdf")
#loader = PyPDFLoader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#loader = PyPDFLoader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    add_start_index = False,
    )

pages = loader.load_and_split()
chunks = text_splitter.split_documents(pages)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
vectordb = FAISS.from_documents(chunks, embedding=embeddings)

In [12]:
query = "Responde en español: ¿Cuál es la experiencia de la docente?"
docs = vectordb.similarity_search(query, k=5)
#docs = loaded_VectorStore.max_marginal_relevance_search(query=query, k=5)

llm = LLM_gradioAPI(n=2000) # hugging face repo
llm.client_api = "https://ysharma-explore-llamav2-with-tgi.hf.space/"
llm.api_name = "/chat_1"

# Build prompt
template = """Use the following pieces of context to answer the question at the end. Use the same language as the Question. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


chain = load_qa_chain(
    llm=llm, 
    chain_type="stuff", 
    verbose=False,
    prompt=QA_CHAIN_PROMPT
    )
#response = chain.run(input_documents=docs, question=query)
#with get_openai_callback() as cb:
#    response = chain.run(input_documents=docs, question=query)
#    print(cb)

response = chain.run(input_documents=docs, question=query)
print(response)
print("\nReferencias:")
for doc in docs:
    print("\nPágina no.: " + str(doc.metadata["page"]) + "\n", doc.page_content[:150] + "...")

Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
La docente tiene experiencia en el área comercial y ventas de empresas de servicio como Principal Créditos Hipotecarios, Andueza Corredores de Seguros, Santiago - Chile. Además, ha trabajado en la gerencia de ventas de Joyerías Anahí, Santa Cruz - Bolivia. También ha sido consultora en temas empresariales y de marketing. Además, ha sido docente en pre y postgrado en UTEPSA, UAGRM Business School y UCB.

Referencias:

Página no.: 7
 o Contenido Práctico. Se basa en la aplicación del contenido teórico en los trabajos desarrollados por los 
postgraduantes así como  el estudio artícu...

Página no.: 1
 Experiencia en el área comercial y ventas de empresas de servicio como Principal Créditos Hipotecarios, 
Andueza Corredores de Seguros, Santiago -Chil...

Página no.: 7
 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   
  E-mail: solicitudes@uagrmbs.edu.bo   
  
  ...

P

### Second draft

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings # see also sentence-transformers
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from src import LLM_gradioAPI
from langchain.document_loaders import PyPDFLoader # install pypdf
from langchain.prompts import PromptTemplate

loader = PyPDFLoader("PROGRAMA ANALITICO MÓDULO.pdf")
#loader = PyPDFLoader("PROGRAMA ANALITICO MÓDULO.pdf")
#loader = PyPDFLoader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#loader = PyPDFLoader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    add_start_index = False,
    )

pages = loader.load_and_split()
chunks = text_splitter.split_documents(pages)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
vectordb = FAISS.from_documents(chunks, embedding=embeddings)

In [49]:
query = "Responde en español: ¿Cuál es la experiencia de la docente?"
docs = vectordb.similarity_search(query, k=5)
#docs = loaded_VectorStore.max_marginal_relevance_search(query=query, k=5)

llm = LLM_gradioAPI(n=2000) # hugging face repo
llm.client_api = "https://ysharma-explore-llamav2-with-tgi.hf.space/"
llm.api_name = "/chat_1"

# Build prompt
template = """Use the following pieces of context to answer the question at the end. Use the same language as the Question. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    #memory = memory,
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 5}),
    return_source_documents=True,
    chain_type="stuff",
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)
print(result["result"])
result["source_documents"][0].metadata

Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
La docente tiene experiencia en el área comercial y ventas de empresas de servicio, como Principal Créditos Hipotecarios, Andueza Corredores de Seguros, y Joyerías Anahí. Además, ha trabajado en la gerencia de ventas de Joyerías Anahí y ha sido consultora en temas empresariales y de marketing. También ha sido docente en pre y postgrado en UTEPSA, UAGRM Business School y UCB.


{'source': 'PROGRAMA ANALITICO MÓDULO.pdf', 'page': 7}

In [1]:

from langchain.chains import ConversationalRetrievalChain

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 5}),
    return_source_documents=True,
    return_generated_question=True,
)

result = qa_chain({"question": query, "chat_history" : []})

print(result["answer"])
result["source_documents"][0].metadata

NameError: name 'llm' is not defined

See memory + source doc
also, whats the difference between RetrievalQA.from_chain_type and ConversationalRetrievalChain.from_llm

In [37]:

chain = load_qa_chain(
    llm=llm, 
    chain_type="stuff", 
    verbose=False,
    prompt=QA_CHAIN_PROMPT,
    #memory=memory
    )
#response = chain.run(input_documents=docs, question=query)
#with get_openai_callback() as cb:
#    response = chain.run(input_documents=docs, question=query)
#    print(cb)

response = chain.run(input_documents=docs, question=query)
print(response)
print("\nReferencias:")
for doc in docs:
    print("\nPágina no.: " + str(doc.metadata["page"]) + "\n", doc.page_content[:150] + "...")

Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
La docente tiene experiencia en el área comercial y ventas de empresas de servicio como Principal Créditos Hipotecarios, Andueza Corredores de Seguros, Santiago - Chile. Además, ha trabajado en la gerencia de ventas de Joyerías Anahí, Santa Cruz - Bolivia. También ha sido consultora en temas empresariales y de marketing. Además, ha sido docente en pre y postgrado en UTEPSA, UAGRM Business School y UCB.

Referencias:

Página no.: 7
 o Contenido Práctico. Se basa en la aplicación del contenido teórico en los trabajos desarrollados por los 
postgraduantes así como  el estudio artícu...

Página no.: 1
 Experiencia en el área comercial y ventas de empresas de servicio como Principal Créditos Hipotecarios, 
Andueza Corredores de Seguros, Santiago -Chil...

Página no.: 7
 UAGRM  Business School    Calle Gral. Saavedra Nº 137     Teléfono  3 -366814      www.uagrmbs.edu.bo   
  E-mail: solicitudes@uagrmbs.edu.bo   
  
  ...

P

### A third draft

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings # see also sentence-transformers
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
from src import LLM_gradioAPI
from langchain.document_loaders import PyPDFLoader # install pypdf
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

loader = PyPDFLoader("PROGRAMA ANALITICO MÓDULO.pdf")
#loader = PyPDFLoader("PROGRAMA ANALITICO MÓDULO.pdf")
#loader = PyPDFLoader("Philip Kotler, Gary Armstrong - Marketing_ Versión latinoamerica (2007).pdf")
#loader = PyPDFLoader("Philip Kotler, Gary Armstrong, Sridhar Balasubramanian - Principles of Marketing-Pearson (2023).pdf")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    add_start_index = False,
    )

pages = loader.load_and_split()
chunks = text_splitter.split_documents(pages)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v1")
vectordb = FAISS.from_documents(chunks, embedding=embeddings)

c:\Users\anton_glfk00f\miniconda3\envs\llm_app_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
query = "¿Cuál es la experiencia de la docente?"

# Define the LLM endpoint
llm = LLM_gradioAPI(n=2000) # hugging face repo
llm.client_api = "https://ysharma-explore-llamav2-with-tgi.hf.space/"
llm.api_name = "/chat_1"

# Build prompt
template = """Use the following pieces of context to answer the question\
    at the end. Use the same language as the Question. If you don't know\
    the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Define memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    #return_messages=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    chain_type="stuff", 
    retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 5}),
    return_source_documents=True,
    return_generated_question=True,
)

result = qa_chain({"question": query, "chat_history" : []})
response = result["answer"]
source_docs = result["source_documents"]

print(response)
print("\nReferencias:")
for doc in source_docs:
    print("\nPágina no.: " + str(doc.metadata["page"]) + "\n", doc.page_content[:150] + "...")


Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
La docente tiene experiencia en el área comercial y ventas de empresas de servicio, como Principal Créditos Hipotecarios, Andueza Corredores de Seguros, y Joyerías Anahí. Además, ha trabajado como consultora en temas empresariales y de marketing, y ha impartido clases en pre y postgrado en varias universidades, incluyendo la UTEPSA, la UAGRM Business School y la UCB.

Referencias:

Página no.: 7
 o Contenido Práctico. Se basa en la aplicación del contenido teórico en los trabajos desarrollados por los 
postgraduantes así como  el estudio artícu...

Página no.: 1
 Experiencia en el área comercial y ventas de empresas de servicio como Principal Créditos Hipotecarios, 
Andueza Corredores de Seguros, Santiago -Chil...

Página no.: 2
 Actividades del capítulo  
Lecturas Básicas:  o   
• Libro MARKETING (KOTLER Y ARMSTRONG) CAP. 1.   
• Libro MARKETING (KOTLER Y ARMSTRONG) CAP. 2.   ...

Página no.: 4
 UAGRM  Business School

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings # see also sentence-transformers
from src import LLM_gradioAPI
from langchain.document_loaders import PyPDFLoader # install pypdf
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain # similar to RetrievalQA but with memory capabilities

def qa_convertational_chain_function(
        file,
        query,
        llm,
        chain_type,
        retreiver_search_type,
        retreiver_k,
        ):
    
    """
    Returns ConversationalRetrievalChain object.
    """
    
    query = query

    # Build prompt
    template = """Use the following pieces of context to answer the question\
        at the end. Use the same language as the Question. If you don't know\
        the answer, just say that you don't know, don't try to make up an answer.
    {context}
    Question: {question}
    Helpful Answer:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
    
    # Creating vector db
    loader = PyPDFLoader(file)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        add_start_index = False,
        )
    pages = loader.load_and_split()
    chunks = text_splitter.split_documents(pages)
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/distiluse-base-multilingual-cased-v1",
        cache_folder="./sentence_transformers"
        )
    vectordb = FAISS.from_documents(chunks, embedding=embeddings)

    retreiver = vectordb.as_retriever(
        search_type=retreiver_search_type, 
        search_kwargs={"k": retreiver_k}
        )

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm, 
        chain_type=chain_type, 
        retriever=retreiver,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa_chain

In [ ]:
# add openai key functionality

In [ ]:
input = {
  "question": "What is the capital of France?",
  "chat_history": [
    {"role": "user", "content": "Hello!"},
    {"role": "assistant", "content": "Hi there!"}
  ]
}

In [4]:
memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='Responde en español: ¿qué enseña la docente?', additional_kwargs={}, example=False),
 AIMessage(content='La docente enseña temas relacionados con la gestión de marketing, incluyendo el análisis del entorno de marketing, la segmentación de mercados, y la toma de decisiones estratégicas. Ella utiliza una metodología participativa y discutida, donde los postgraduados tienen la oportunidad de aprender a través de la discusión de casos de estudio y la aplicación de teorías empresariales en situaciones simuladas. La docente también proporciona recursos académicos de apoyo y espera que los postgraduados hayan estudiado los temas correspondientes con anticipación para poder aprovechar al máximo las clases.', additional_kwargs={}, example=False)]

### Final draft

In [1]:
# Setup
from src import LLM_gradioAPI
from src import qa_convertational_chain_function

# Define the LLM endpoint
llm = LLM_gradioAPI(n=2000) # hugging face repo
llm.client_api = "https://ysharma-explore-llamav2-with-tgi.hf.space/"
llm.api_name = "/chat_1"

file = "PROGRAMA ANALITICO MÓDULO.pdf"
chain_type = "stuff"
retreiver_search_type = "mmr"
retreiver_k = 5

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
response = []
qa_chain = qa_convertational_chain_function(file, llm, chain_type, retreiver_search_type, retreiver_k)


c:\Users\anton_glfk00f\miniconda3\envs\llm_app_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# todo: summarize the first 30 pages of doc

In [2]:
query = "Resume tu anterior respuesta"

result = qa_chain({
    "question": query,
    "chat_history" : memory.load_memory_variables({})["chat_history"]
    })
response = result["answer"]
source_docs = result["source_documents"]

memory.save_context({"input": query}, {"output": response})

# to do: add reset memory in application

print(response)
print("\nReferencias:")
for doc in source_docs:
    print("\nPágina no.: " + str(doc.metadata["page"]) + "\n", doc.page_content[:150] + "...")

Loaded as API: https://ysharma-explore-llamav2-with-tgi.hf.space/ ✔
Sure, I can summarize my previous response for you.

The module "Gestión de Marketing" aims to provide students with the methodology and tools to understand the market in which businesses compete, and to develop the skills to analyze, diagnose, and make strategic decisions in marketing. The module covers topics such as segmentation, target market selection, and marketing mix.

The objectives of the module are to enable students to:

1. Understand the importance of marketing and its role in business strategy.
2. Analyze the market and identify market segments.
3. Develop a marketing plan and implement it effectively.
4. Evaluate the effectiveness of marketing strategies and tactics.

The module includes case studies and practical exercises to help students apply theoretical concepts to real-world scenarios. The assessment includes participation, practical works, and a final exam.

The module is designed to help students

### Summarization

In [ ]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain

llm = ChatOpenAI(temperature=0)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)